In [1]:
import tensorflow as tf

In [21]:
def get_data(url, name):
    import urllib
    import zipfile
    import os

    if not os.path.isdir(name): # check whether the directory exists or not
        if not os.path.isfile(name): # check whether zip is downloaded or not
            try: 
                urllib.request.urlretrieve(url, name)
            except:
                pass

        if name[-4:] == ".zip" or name[-4:] == ".rar":
            zip_ref = zipfile.ZipFile(name, "r")
            zip_ref.extractall()
            zip_ref.close()

In [24]:
url = "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
get_data(url, "nlp_getting_started.zip")

In [23]:
url = "https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py"
get_data(url, "helper_functions.py")

from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

In [25]:
# Visualizing a text dataset 
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [26]:
# shuffle training dataframe
train_df = train_df.sample(frac=1, random_state=42)

In [27]:
train_df.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [31]:
# split data into training and validation sets
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                           train_df["target"].to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)